## 画像URLから画像を取得する

In [22]:
import sys
import os
import shutil
import requests
sys.path.append('..')
from WebScrapingTool import Base_UserFunction as uf
from WebScrapingTool import Base_GetHtmlList as _gethtml
import glob
import json

def getImage(image_url, savePass, fn):
    try:
        response = requests.get(image_url, stream=True, headers={'User-agent': 'Mozilla/5.0'})
        if response.status_code == 200:
            #画像URLであるのでtypeを気にせず取得する。
            filename = savePass + '/' + fn
            with open(filename, 'wb') as file:
                response.raw.decode_content = True
                shutil.copyfileobj(response.raw, file)
            #print('Success')

        else:
            print('ERROR  response.status_code: ' + str(response.status_code))
            image_url = image_url.replace('/originals/','/736x/')
            print('TryAgain url:' + image_url)
            response = requests.get(image_url, stream=True, headers={'User-agent': 'Mozilla/5.0'})
            if response.status_code == 200:
                filename = savePass + '/' + fn
                with open(filename, 'wb') as file:
                    response.raw.decode_content = True
                    shutil.copyfileobj(response.raw, file)
                #print('Success')
            else:
                print('ERROR  response.status_code: ' + str(response.status_code))
                image_url = image_url.replace('/originals/','/564x/')
                print('TryAgain url:' + image_url)
                response = requests.get(image_url, stream=True, headers={'User-agent': 'Mozilla/5.0'})
                if response.status_code == 200:
                    filename = savePass + '/' + fn
                    with open(filename, 'wb') as file:
                        response.raw.decode_content = True
                        shutil.copyfileobj(response.raw, file)
                        #print('Success')

            
            
    except:
        print('ERROR  url: ' + image_url)
        
    #print('\n')
        
def main():
    import re
    print("\n[Start]"  + uf.getNowTime() + '\n')

    #設定ファイルから必要な情報を取得する
    #タグ
    tag_saveFolder = '[a]'
    Folder = ''
    try:
        with open('_Setting.txt', mode='r') as f:
            lines = f.readlines()
            for l in lines:
                if l.startswith(tag_saveFolder,0,3):
                    Folder = l.replace(tag_saveFolder,'').rstrip()
    except:
        print('[!!!ERROR!!!] Read Setting.text')
        return        
    
    if len(Folder) <= 0:
        print('[!!!ERROR!!!] Image data storage folder is None!')
        return  
    
    #画像URLを抜き出したテキストから画像を取得する
    baseFolder = glob.glob(Folder)
    
    for fd_path, sb_folder, sb_file in os.walk(Folder):
        for fol in sb_folder:
            path = Folder + '/' + fol
            print('\n++++++++++\n[SaveFolder]' + path + '\n++++++++++\n')

            #バックアップファイルに追加
            backup = path + '/_Backup_' + fol + '.txt'
            with open(backup, mode='a') as fb: #バックアップファイル　追記モード
                #日付で始まるテキストのみ対象とする
                list = glob.glob( path + '/[0-9]*.txt')

                for fn in list:
                    #ファイルを開く
                    with open(fn, mode='r') as f:
                        cnt = 0
                        for line in f:
                            di = json.loads(line)
                            #print(di)
                            ImgFileName = di['name']
                            URL = di['url']
                            getImage(URL,path,ImgFileName)
                            cnt += 1
                            print("...Access ImageURL : " + URL + '  ' + str(cnt))
                            uf.fileWrite(fb, line)
                        print('\n...Get Size :' + str(cnt) + '\n')

                    #ファイル削除    
                    os.remove(fn) 
                    
            uf.fileDataSlim(backup)   
    
    print("\n[ End ]"  + uf.getNowTime() + '\n')
    
    
if __name__ == '__main__':
    main()


[Start]20181030171404


++++++++++
[SaveFolder]#Image/#Free_20181030164513
++++++++++


++++++++++
[SaveFolder]#Image/#Free_20181030164557
++++++++++


++++++++++
[SaveFolder]#Image/#Free_20181030164713
++++++++++


[ End ]20181030171404

